# 第9章 面向对象

## 9.1 基础知识

### 9.1.1 私有属性

在Python圈有句话：`We are all consenting adults here`；对于不想外部访问使用的属性，请在命名前加单下划线.双下划线是一种自私行为。

在模块中单下划线的变量，在from mymod import * 时不会被导入，但是from mymod import _mood可以导入

In [2]:
class Person:
    def __init__(self, name, gender):
        self._name = name
        self.__gender = gender

wang = Person("小王", "男")
wang.__dict__

{'_name': '小王', '_Person__gender': '男'}

### 9.1.2 类方法

类实例化后，才能调用方法。能不能直接调用呢？可以，用`@classmethod`装饰方法，这么做一般是用在：定义备选__init__ 方法。

⚠️注意，第一个参数要传入类本身，约定写法是`cls`

In [6]:
class Person:
    def __init__(self, name, gender):
        self._name = name
        self._gender = gender
    
    def __repr__(self):
        return f"name:{self._name}; gender:{self._gender}"
        
    @classmethod
    def male(cls, name):
        return cls(name, "男")
    
Person.male("李四")

name:李四; gender:男

### 9.1.3 静态方法

如果这个方法和类的状态属性(`__dict__`)没什么关系，那么就请用 静态方法`staticmethod`吧。

In [8]:
class Person:
    def __init__(self, name, gender):
        self._name = name
        self._gender = gender
    
    def eat():
        print("所有人都会吃饭")
    
Person.eat()

所有人都会吃饭


### 9.1.4 属性与方法

类的方法是可以转化为方法。`@property`装饰的函数，可以像属性一样访问，同时可以对属性的`get set del` 作自定义设置。

In [4]:
from enum import Enum

class Gender(Enum):
    Female = 0
    Male = 1

class Person:
    def __init__(self, name, gender):
        self._name = name
        self._gender = gender
    
    def __repr__(self):
        return f"I'm a Human!"
    
    @property
    def trans(self):
        self._gender = Gender(abs(1-self._gender.value))
        return self._gender
    
     
zhao = Person("老赵", Gender.Male)
# 像调用属性一样，调用方法。
zhao.trans

<Gender.Female: 0>

### 9.1.5 鸭子类型

在纯粹的鸭子类型编程风格下（只关注行为，而不关注对象的类型），不应该出现任何的`isinstance()`类型判断，你可以用EAFP风格代码操作，也可以用`hasattr(object, "foo")` raise 判断。

In [41]:
hasattr(Person, "__repr__")

True

## 9.2 抽象类

自己定义抽象基类，要继承`abc.ABC`；但不要在生产代码中定义抽象基类，滥用抽象基类会造成灾难性后果，表明语言太注重表面形式。99%的情况下只需要正确使用现有的抽象基类，就够用。

抽象类的出现，让`isinstance()`没有那么尴尬了。传统模式下`isinstance(obj, BaseObj)`判断对象是否属于特定类型。但它和鸭子类型理念相悖，直到Python2.6推出的抽象类。

抽象类作为一种特殊的基类，为我们提供了更为灵活的子类化机制. `__subclasshook__`是被装饰器装饰的类方法，当使用`isinstance`检查对象是否属于某个抽象类时，如果后者定义了这个方法，那么该方法就会被处罚。

`C.__mro__`代表C继承的父类有哪些，C3算法实现。

像下面这种只关心结构，不关心真实继承关系的，被称为“结构化子类”。

In [11]:
from abc import ABC


class Validator(ABC):
    
    @classmethod
    def __subclasshook__(cls, C):
        if any("trans" in B.__dict__ for B in C.__mro__):
            return True
        return NotImplemented
 
isinstance(zhao, Validator)

True

除了使用`__subclasshook__`方法，我们还可以使用`register()`方法，手动注册一个类为另一个类的子类：

In [9]:
class Foo:
    pass

print("两者的关系：", issubclass(Foo,Validator))
Validator.register(Foo)
print("两者的关系：", issubclass(Foo,Validator))

两者的关系： False
两者的关系： True


### 9.2.1 抽象方法

用`@abstractmethod` 装饰类方法后，继承此类的类，必须实现被装饰的类方法

In [62]:
from abc import ABC, abstractmethod


class Validator(ABC):
    
    @abstractmethod
    def validate(self, value):
        raise NotImplementedError
        
        
class MyValidate(Validator):
    pass

obj = MyValidate()

TypeError: Can't instantiate abstract class MyValidate with abstract method validate

### 9.2.2 多重继承与MRO

Python支持多重继承，这让寻找父类更加复杂，其算法既不是广度优先，也不是深度优先：

In [64]:
class A:
    def say(self):
        print("Im A")
        
class B(A):
    def say(self):
        print("Im B")
        
class C(A):
    def say(self):
        print("Im C")
        
class D(B,C):
    def say(self):
        print("Im D")
        

D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]

上面的例子看起来像是广度优先？其实不然：

In [66]:
class A:
    def say(self):
        print("Im A")
        
class B:
    def say(self):
        print("Im B")
        
class C(A):
    def say(self):
        print("Im C")
        
class D(B):
    def say(self):
        print("Im D")

class E(C,D):
    def say(self):
        print("Im D")

E.mro()

[__main__.E, __main__.C, __main__.A, __main__.D, __main__.B, object]

### 9.2.3 super()

`super()`的真实含义，并不是直接调用父类的方法，而是MRO链条的下一个类。看下面的例子，如果真是直接调用其父类，那么就应该打印：`D->B->A`

In [72]:
class A:
    def __init__(self):
        print("Im A")
        
class B(A):
    def __init__(self):
        print("Im B")
        super().__init__()
        
class C(A):
    def __init__(self):
        print("Im C")
        
class D(B,C):
    def __init__(self):
        print("Im D")
        super().__init__()
        
d = D()

Im D
Im B
Im C


继承，是一件会让代码紧耦合的行为。要非常避免这样做。要对事物的行为建模，而不是对事物本身建模。把一些类行为直接定义为类，然后多使用组合，而不是继承。

### 9.2.4 单例模式

一般来说，单例模式可以用`__new__`方法来实现，还有种方法是`预绑定方法模式`：因为Python在执行`import` 导入模块时，被导入的模块会在内存中只保存一份。因此只需要在模块中创建一个全局对象：


In [73]:
# 普通的单例模式
class Single:
    """程序配置类，使用单例模式"""
    
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            inst = super().__new__(cls, *args, **kwargs)
            _instance = inst
        return cls._instance
    def __init__(self):
        pass
    
single1 = Single()
single2 = Single()
print(id(single1) == id(single2))

True


In [ ]:
# 利用模块特性

class Single:
    
    def __init__(self):
        pass
    def foo1(self):
        pass 
    def foo1(self):
        pass 
    
_single = Single()
something = _single.foo1


# 在需要用到它的地方
from utils import something


## 9.3 类型注解

类型注解是一种给函数参数、返回值一集任何变量增加类型描述的技术。规范的注解可以大大提升代码可读性。`typing`常用的有：
- `List`
- `Dict`
- `Callable`
- `TextIO`
- `Any`

引入静态检查后，除了增加可读性，配合`mypy`还能提高代码正确性。

Python3.8 中新增了`Protocol`类型，比抽象类更加接近传统意义上的接口。

In [ ]:
from typing import List
import random

class Duck:
    def __init__(self, color: str):
        self.color = color
        

def create_ducks(number:int)-> List[Duck]:
    # 声明变量的时候也可以加上注解
    ducks: List[Duck] = []
    for _ in number:
        color = random.choice(['yello', 'white', 'black'])
        ducks.append(Duck(color=color))
    return ducks